## Runtime Environment

* python >= 3.7
* pytorch >= 1.0
* transformers
* pandas
* nltk
* numpy
* sklearn
* pickle
* tqdm
* json

## Set Random seed

In [1]:
import torch
torch.manual_seed(58)

# Take a  view of dataset

In [2]:
import pandas as pd

dataset = pd.read_csv('../data/task2_trainset.csv', dtype=str)
dataset.head()

,Id,Title,Abstract,Authors,Categories,Created Date,Task 2
0,D00001,A Brain-Inspired Trust Management Model to Ass...,Rapid popularity of Internet of Things (IoT) a...,Mahmud/Kaiser/Rahman/Rahman/Shabut/Al-Mamun/Hu...,cs.CR/cs.AI/q-bio.NC,2018-01-11,THEORETICAL
1,D00002,On Efficient Computation of Shortest Dubins Pa...,"In this paper, we address the problem of compu...",Sadeghi/Smith,cs.SY/cs.RO/math.OC,2016-09-21,THEORETICAL
2,D00003,Data-driven Upsampling of Point Clouds,High quality upsampling of sparse 3D point clo...,Zhang/Jiang/Yang/Yamakawa/Shimada/Kara,cs.CV,2018-07-07,ENGINEERING
3,D00004,Accessibility or Usability of InteractSE? A He...,Internet is the main source of information now...,Aqle/Khowaja/Al-Thani,cs.HC,2018-08-29,EMPIRICAL
4,D00005,Spatio-Temporal Facial Expression Recognition ...,Automated Facial Expression Recognition (FER) ...,Hasani/Mahoor,cs.CV,2017-03-20,ENGINEERING


**Id**: 流水號  
**Title**: 論文標題  
**Abstract**: 論文摘要內容, 句子間以 **$$$** 分隔  
**Authors**: 論文作者  
**Categories**: 論文類別  
**Created date**: 論文上傳日期  
**Task 2**: 論文分類類別, 若句子有多個類別,以 **空格** 分隔 

# Data processing

## 刪除多於資訊 (Remove redundant information)  
我們在資料集中保留了許多額外資訊供大家使用，但是在這次的教學中我們並沒有用到全部資訊，因此先將多餘的部分先抽走。  
In dataset, we reserved lots of information. But in this tutorial, we don't need them, so we need to discard them.

In [3]:
dataset.drop('Title',axis=1,inplace=True)
dataset.drop('Categories',axis=1,inplace=True)
dataset.drop('Created Date',axis=1, inplace=True)
dataset.drop('Authors',axis=1,inplace=True)

In [4]:
dataset.head()

,Id,Abstract,Task 2
0,D00001,Rapid popularity of Internet of Things (IoT) a...,THEORETICAL
1,D00002,"In this paper, we address the problem of compu...",THEORETICAL
2,D00003,High quality upsampling of sparse 3D point clo...,ENGINEERING
3,D00004,Internet is the main source of information now...,EMPIRICAL
4,D00005,Automated Facial Expression Recognition (FER) ...,ENGINEERING


## 資料切割  (Partition)
在訓練時，我們需要有個方法去檢驗訓練結果的好壞，因此需要將訓練資料切成training/validataion set。   
While training, we need some method to exam our model's performance, so we divide our training data into training/validataion set.

In [5]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

trainset, validset = train_test_split(dataset, test_size=0.1, random_state=42)

trainset.to_csv('trainset.csv', index=False)
validset.to_csv('validset.csv', index=False)

### For test data

In [6]:
dataset = pd.read_csv('../data/task2_public_testset.csv', dtype=str)
dataset.drop('Title',axis=1,inplace=True)
dataset.drop('Categories',axis=1,inplace=True)
dataset.drop('Created Date',axis=1, inplace=True)
dataset.drop('Authors',axis=1,inplace=True)
dataset.to_csv('testset.csv',index=False)

### 資料格式化 (Data formatting)  
有了字典後，接下來我們要把資料整理成一筆一筆，把input的句子轉成數字，把答案轉成onehot的形式。  
這裡，我們一樣使用`multiprocessing`來加入進行。  
After building dictionary, that's mapping our sentences into number array, and convert answers to onehot format.  

# BERT

In [7]:
from transformers import BertTokenizer, BertForMaskedLM

PRETRAINED_MODEL_NAME = 'bert-base-cased'

NUM_LABELS = 3

In [8]:
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

vocab = tokenizer.vocab
print("字典大小：", len(vocab))

字典大小： 28996


In [9]:
from tqdm import tqdm_notebook as tqdm
from multiprocessing import Pool

def label_to_onehot(labels):
    """ Convert label to onehot .
        Args:
            labels (string): sentence's labels.
        Return:
            outputs (onehot list): sentence's onehot label.
    """
    label_dict = {'THEORETICAL': 0, 'ENGINEERING':1, 'EMPIRICAL':2}
    onehot = [0,0,0]
    for l in labels.split():
        if l == "OTHERS":
            continue
        onehot[label_dict[l]] = 1
    return onehot
        
def sentence_to_indices(sent, tokenizer):
    """ Convert sentence to its word indices.
    Args:
        sentence (str): One string.
    Return:
        indices (list of int): List of word indices.
    """
    return [tokenizer.convert_tokens_to_ids(word) for word in tokenizer.tokenize(sent)]
    
def get_dataset(data_path, tokenizer, n_workers=4):
    """ Load data and return dataset for training and validating.

    Args:
        data_path (str): Path to the data.
    """
    dataset = pd.read_csv(data_path, dtype=str)

    results = [None] * n_workers
    with Pool(processes=n_workers) as pool:
        for i in range(n_workers):
            batch_start = (len(dataset) // n_workers) * i
            if i == n_workers - 1:
                batch_end = len(dataset)
            else:
                batch_end = (len(dataset) // n_workers) * (i + 1)
            
            batch = dataset[batch_start: batch_end]
            results[i] = pool.apply_async(preprocess_samples, args=(batch, tokenizer))

        pool.close()
        pool.join()

    processed = []
    for result in results:
        processed += result.get()
    return processed

def preprocess_samples(dataset, tokenizer):
    """ Worker function.

    Args:
        dataset (list of dict)
    Returns:
        list of processed dict.
    """
    processed = []
    for sample in tqdm(dataset.iterrows(), total=len(dataset)):
        processed.append(preprocess_sample(sample[1], tokenizer))

    return processed

def preprocess_sample(data, tokenizer):
    """
    Args:
        data (dict)
    Returns:
        dict
    """
    processed = {}
    processed['tokens'] = [sentence_to_indices(sent, tokenizer) 
                           for sent in data['Abstract'].split('$$$')]
    processed['tokens'] = sum(processed['tokens'], [])
#     processed['tokens'] = [tokenizer.convert_tokens_to_ids('[CLS]')] + processed['tokens'] + [tokenizer.convert_tokens_to_ids('[SEP]')]
    processed['segments'] = [0] * len(processed['tokens'])
    
    if 'Task 2' in data:
        processed['Label'] = label_to_onehot(data['Task 2'])
        
    return processed

In [10]:
print('[INFO] Start processing trainset...')
train = get_dataset('trainset.csv', tokenizer, n_workers=8)
print('[INFO] Start processing validset...')
valid = get_dataset('validset.csv', tokenizer, n_workers=8)
print('[INFO] Start processing testset...')
test = get_dataset('testset.csv', tokenizer, n_workers=8)

[INFO] Start processing trainset...








[INFO] Start processing validset...








[INFO] Start processing testset...










In [11]:
# max_len sentence
m = 0
for t in test:
    if len(t['tokens']) > m:
        m = len(t['tokens'])
print(m)

709


## 資料封裝 (Data packing)

為了更方便的進行batch training，我們將會借助[torch.utils.data.DataLoader](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader)。  
而要將資料放入dataloader，我們需要繼承[torch.utils.data.Dataset](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset)，撰寫適合這份dataset的class。  
`collate_fn`用於batch data的後處理，在`dataloder`將選出的data放進list後會呼叫collate_fn，而我們會在此把sentence padding到同樣的長度，才能夠放入torch tensor (tensor必須為矩陣)。  

To easily training in batch, we'll use `dataloader`, which is a function built in Pytorch[torch.utils.data.DataLoader](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader)  
To use datalaoder, we need to packing our data into class `dataset` [torch.utils.data.Dataset](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset)  
`collate_fn` is used for data processing.

In [12]:
from torch.utils.data import Dataset
import torch

class BertDataset(Dataset):
    def __init__(self, data, max_len = 512):
        self.data = data
        self.max_len = max_len
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]
        
    def collate_fn(self, datas):
        # get max length in this batch
        max_len = max([min(len(data['tokens']), self.max_len) for data in datas])
        batch_tokens = []
        batch_segments = []
        batch_masks = []
        batch_label = []
        for data in datas:
            # padding abstract to make them in same length
            abstract_len = len(data['tokens'])
            if abstract_len > max_len:
                batch_tokens.append(data['tokens'][:max_len])
                batch_segments.append(data['segments'][:max_len])
                batch_masks.append([1] * max_len)
            else:
                batch_tokens.append(data['tokens'] + [0] * (max_len-abstract_len))
                batch_segments.append(data['segments'] + [0] * (max_len-abstract_len))
                batch_masks.append([1] * abstract_len  + [0] * (max_len-abstract_len))
            # gather labels
            if 'Label' in data:
                batch_label.append(data['Label'])
        return torch.LongTensor(batch_tokens), torch.LongTensor(batch_segments), torch.LongTensor(batch_masks), torch.FloatTensor(batch_label)

In [13]:
trainData = BertDataset(train)
validData = BertDataset(valid)
testData = BertDataset(test)

# model

In [14]:
from transformers import BertModel, BertPreTrainedModel

In [15]:
class BertForMultiLabelSequenceClassification(BertPreTrainedModel):
    """BERT model for classification.
    This module is composed of the BERT model with a linear layer on top of
    the pooled output.
    """
    def __init__(self, config):
        super(BertForMultiLabelSequenceClassification, self).__init__(config)
        
        self.bert = BertModel(config)
        self.dropout = torch.nn.Dropout(0.1)
        self.linear = torch.nn.Linear(config.hidden_size, config.hidden_size)
        self.classifier = torch.nn.Linear(config.hidden_size, config.num_labels)
        self.sigmoid = torch.nn.Sigmoid()
        
        self.init_weights()

    def forward(self, input_ids, token_type_ids=None, attention_mask=None, 
                position_ids=None, head_mask=None, labels=None):
        
        outputs = self.bert(input_ids, 
                            token_type_ids = token_type_ids, 
                            attention_mask = attention_mask, 
                            position_ids=None, head_mask=None)
        
        pooled_output = outputs[1]
        
        pooled_output = self.dropout(pooled_output)
        logits = self.linear(pooled_output)
#         logits = self.dropout(logits)
        logits = self.classifier(logits)
#         logits = self.sigmoid(logits)
        
        outputs = (logits,) + outputs[1:]
        
        return outputs[0]
        
    def freeze_bert_encoder(self):
        for param in self.bert.parameters():
            param.requires_grad = False
    
    def unfreeze_bert_encoder(self):
        for param in self.bert.parameters():
            param.requires_grad = True

In [16]:
model = BertForMultiLabelSequenceClassification.from_pretrained(PRETRAINED_MODEL_NAME, num_labels = NUM_LABELS)
# model.freeze_bert_encoder()

In [17]:
print(model)

BertForMultiLabelSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-1

# Train

指定使用的運算裝置  
Designate running device.

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

定義一個算分公式, 讓我們在training能快速了解model的效能
Define score function, let us easily observe model performance while training.

In [19]:
class F1():
    def __init__(self):
        self.threshold = 0.0
        self.n_precision = 0
        self.n_recall = 0
        self.n_corrects = 0
        self.name = 'F1'

    def reset(self):
        self.n_precision = 0
        self.n_recall = 0
        self.n_corrects = 0

    def update(self, predicts, groundTruth):
        predicts = predicts > self.threshold
        predicts = predicts.type(torch.float32) 
        predicts = self.add_labels(predicts)
        groundTruth = self.add_labels(groundTruth)
        self.n_precision += torch.sum(predicts).data.item()
        self.n_recall += torch.sum(groundTruth).data.item()
        self.n_corrects += torch.sum(groundTruth * predicts).data.item()

    def add_labels(self, labels):
        new_axis = []
        for label in labels:
            new_axis.append([1]) if torch.sum(label) == 0 else new_axis.append([0])
        return torch.cat((labels, torch.Tensor(new_axis)), 1)
        
    def get_score(self):
        recall = self.n_corrects / self.n_recall
        precision = self.n_corrects / (self.n_precision + 1e-20)
        return 2 * (recall * precision) / (recall + precision + 1e-20)

    def print_score(self):
        score = self.get_score()
        return '{:.5f}'.format(score)


## 定義參數

In [20]:
BATCH_SIZE = 2
EPOCHS = 6

In [21]:
import os
def _run_epoch(epoch, training):
    model.train(training)
    if training:
        description = 'Train'
        BATCH_SIZE = 2
        dataset = trainData
        shuffle = True
    else:
        description = 'Valid'
        BATCH_SIZE = 2
        dataset = validData
        shuffle = False
    dataloader = DataLoader(dataset=dataset,
                            batch_size=BATCH_SIZE,
                            shuffle=shuffle,
                            collate_fn=dataset.collate_fn,
                            num_workers=4)
    
    trange = tqdm(enumerate(dataloader), total=len(dataloader), desc=description)
    loss = 0.0
    f1_score = F1()
    for i, (tokens, segments, masks, labels) in trange:
        o_labels, batch_loss = _run_iter(tokens, segments, masks, labels)
        
        if training:
            opt.zero_grad()
            batch_loss.backward()
            opt.step()
        
        
        loss += batch_loss.item()
        f1_score.update(o_labels.cpu(), labels)
        
        trange.set_postfix(
            loss=loss / (i + 1), f1=f1_score.print_score())
    if training:
        history['train'].append({'f1':f1_score.get_score(), 'loss':loss/ len(trange)})
    else:
        history['valid'].append({'f1':f1_score.get_score(), 'loss':loss/ len(trange)})

        
def _run_iter(tokens, segments, masks, labels):
    tokens = tokens.to(device)
    segments = segments.to(device)
    masks = masks.to(device)
    labels = labels.to(device)
    outputs = model(tokens, token_type_ids = segments, attention_mask=masks)
    l_loss = criteria(outputs, labels)
    return outputs, l_loss

def save(epoch):
    if not os.path.exists('model'):
        os.makedirs('model')
    torch.save(model.state_dict(), 'model/model-cased.pkl.'+str(epoch))
    with open('model/history-cased.json', 'w') as f:
        json.dump(history, f, indent=4)

In [22]:
from torch.utils.data import DataLoader
from tqdm import trange
import json

opt = torch.optim.AdamW(model.parameters(), lr=1e-5, eps = 1e-8)

criteria = torch.nn.BCEWithLogitsLoss()
model.to(device)
history = {'train':[],'valid':[]}

In [23]:
# list(model.parameters())[-2]

In [ ]:
for epoch in range(EPOCHS):
    print('Epoch: {}'.format(epoch))
    _run_epoch(epoch, True)
    _run_epoch(epoch, False)
    save(epoch)

Epoch: 0


# Plot

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

with open('model/history-cased.json', 'r') as f:
    history = json.loads(f.read())
    
train_loss = [l['loss'] for l in history['train']]
valid_loss = [l['loss'] for l in history['valid']]
train_f1 = [l['f1'] for l in history['train']]
valid_f1 = [l['f1'] for l in history['valid']]

plt.figure(figsize=(7,5))
plt.title('Loss')
plt.plot(train_loss, label='train')
plt.plot(valid_loss, label='valid')
# plt.xticks(np.arange(0,20))
plt.legend()
plt.show()

plt.figure(figsize=(7,5))
plt.title('F1 Score')
plt.plot(train_f1, label='train')
plt.plot(valid_f1, label='valid')
plt.legend()
plt.show()

print('Best F1 score ', max([[l['f1'], idx] for idx, l in enumerate(history['valid'])]))

# Prediction

In [ ]:
model.load_state_dict(torch.load('model/model-cased.pkl.{}'.format(4)))
model.train(False)
# _run_epoch(1, False)
dataloader = DataLoader(dataset=testData,
                            batch_size=16,
                            shuffle=False,
                            collate_fn=testData.collate_fn,
                            num_workers=4)
trange = tqdm(enumerate(dataloader), total=len(dataloader), desc='Predict')
prediction = []
for i, (tokens, segments, masks, labels) in trange:
    with torch.no_grad():
        o_labels = model(tokens.to(device), segments.to(device), masks.to(device))
        o_labels = o_labels>0.0
        prediction.append(o_labels.to('cpu'))

prediction = torch.cat(prediction).detach().numpy().astype(int)

In [ ]:
def SubmitGenerator(prediction, sampleFile, public=True, filename='prediction-cased.csv'):
    """
    Args:
        prediction (numpy array)
        sampleFile (str)
        public (boolean)
        filename (str)
    """
    sample = pd.read_csv(sampleFile)
    submit = {}
    submit['order_id'] = list(sample.order_id.values)
    redundant = len(sample) - prediction.shape[0]
    if public:
        submit['THEORETICAL'] = list(prediction[:,0]) + [0]*redundant
        submit['ENGINEERING'] = list(prediction[:,1]) + [0]*redundant
        submit['EMPIRICAL'] = list(prediction[:,2]) + [0]*redundant
        submit['OTHERS'] = list(prediction[:,3]) + [0]*redundant
    else:
        submit['THEORETICAL'] = [0]*redundant + list(prediction[:,0])
        submit['ENGINEERING'] = [0]*redundant + list(prediction[:,1])
        submit['EMPIRICAL'] = [0]*redundant + list(prediction[:,2])
        submit['OTHERS'] = [0]*redundant + list(prediction[:,3])
    df = pd.DataFrame.from_dict(submit) 
    df.to_csv(filename,index=False)

In [ ]:
SubmitGenerator(prediction, 
                '../data/task2_sample_submission.csv',
                True, 
                '../../score/bert-cased/bert-cased.csv')